In [157]:
import numpy as np

# mne imports
import mne
from mne import io, read_epochs, concatenate_epochs
from mne.datasets import sample
from mne import Epochs, create_info, events_from_annotations
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [158]:
cd epochs

[Errno 2] No such file or directory: 'epochs'
/Users/jang-wonjun/Documents/eeg/ml/epochs


In [159]:
ls

epoch0-epo.fif   epoch2-epo.fif   epoch5-epo.fif   epoch8-epo.fif
epoch1-epo.fif   epoch3-epo.fif   epoch6-epo.fif   epoch9-epo.fif
epoch10-epo.fif  epoch4-epo.fif   epoch7-epo.fif


In [160]:
n_epochs = 11
epoch_list = [f'epoch{i}-epo.fif' for i in range(n_epochs)]

In [161]:
epoch = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch3-epo.fif ...
    Found the data of interest:
        t =  

Number of events,110
Events,finger_movement: 110
Time range,-2.000 – 2.000 s
Baseline,off


In [204]:
fmin, fmax =2 ,8
epoch_filter = epoch.copy().filter(fmin,fmax)

Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 213 samples (1.664 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 770 out of 770 | elapsed:    0.1s finished


In [205]:
w_tmin, w_tmax = 0, 0.5
on_event = epoch_filter.copy().crop(w_tmin,w_tmax)
off_event = epoch_filter.copy().crop(-2+w_tmin,-2+w_tmax)
        
X = np.concatenate([on_event.get_data(),off_event.get_data()],axis=0)*1000
y = [1 for i in range(X.shape[0]//2)]+[0 for i in range(X.shape[0]//2)]

In [206]:
X.shape # (trials,channels,samples)

(220, 7, 65)

In [207]:
kernels, chans, samples = 1, 7, 65

In [208]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)


In [209]:
X_val,X_test,y_val,y_test = train_test_split(X_test, y_test,test_size=0.5, random_state=42)

In [221]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

In [224]:
y_val.shape,y_test.shape

((22, 2), (22, 2, 2))

In [211]:
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [212]:
numParams    = model.count_params()

In [213]:
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [214]:
class_weights = {0:1, 1:1}

In [222]:
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_val, y_val),
                        callbacks=[checkpointer], class_weight = class_weights)

Epoch 1/300


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/cond/remove_squeezable_dimensions/cond/Squeeze' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/homebrew/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/opt/homebrew/anac

In [216]:
# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

In [217]:
probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

1/1 [==============================] - 0s 53ms/step
Classification accuracy: 0.909091 
